In [2]:
import os
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
import pickle

Using TensorFlow backend.


In [3]:
os.chdir("../")
os.chdir("FinalDataset")

In [4]:
data = pd.read_csv("feature.csv")

In [5]:
data.head()

,Unnamed: 0,File,bodyLength,bscr,dse,dsr,entropy,hasHttp,hasHttps,has_ip,...,numParams,numTitles,num_%20,num_@,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,0,spam_dataset,31,0.064516,0,0,-4.708307,True,False,0,...,0,0,0,0,0.000000,0,2,0.000000,False,81
1,1,spam_dataset,64790,0.306050,0,0,-4.576882,True,False,0,...,0,19,0,0,0.923800,59853,19829,3.018458,False,58
2,2,spam_dataset,757,0.195509,0,0,-4.205536,True,False,0,...,0,1,0,0,0.000000,0,148,0.000000,False,46
3,3,spam_dataset,266310,0.230915,0,0,-4.550931,True,False,0,...,0,648,0,0,0.312647,83261,61495,1.353947,False,52
4,4,spam_dataset,40,0.200000,0,0,-4.689336,True,False,0,...,2,1,0,0,0.000000,0,8,0.000000,False,83


In [6]:
set(data['File'])

{'Benign_list_big_final',
 'Defacement',
 'Malware_dataset',
 'phishing_dataset',
 'spam_dataset'}

In [7]:
data.replace("Benign_list_big_final","Benign",inplace=True)
data.replace("Malware_dataset","Malware",inplace=True)
data.replace("phishing_dataset","Phishing",inplace=True)
data.replace("spam_dataset","Spam",inplace=True)

In [8]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [9]:
data = data.drop(columns = "File")

In [10]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [11]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [14]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Train on 28628 samples, validate on 12270 samples
Epoch 1/50
28628/28628 [==============================] - 3s 98us/step - loss: 1.2577 - accuracy: 0.4623 - val_loss: 1.0429 - val_accuracy: 0.5860
Epoch 2/50
28628/28628 [==============================] - 1s 46us/step - loss: 0.9776 - accuracy: 0.6083 - val_loss: 0.9176 - val_accuracy: 0.6402
Epoch 3/50
28628/28628 [==============================] - 1s 48us/step - loss: 0.8707 - accuracy: 0.6543 - val_loss: 0.8195 - val_accuracy: 0.6738
Epoch 4/50
28628/28628 [==============================] - 1s 47us/step - loss: 0.7861 - accuracy: 0.6887 - val_loss: 0.7510 - val_accuracy: 0.7094
Epoch 5/50
28628/28628 [==============================] - 1s 48us/step - loss: 0.7348 - accuracy: 0.7091 - val_loss: 0.7086 - val_accuracy: 0.7208
Epoch 6/50
28628/28628 [==============================] - 2s 53us/step - loss: 0.6819 - accuracy: 0.7348 - val_loss: 0.6937 - val_accuracy: 0.7256
Epoch 7/50
28628/28628 [==============================] - 1s 41us/st

In [15]:
y_pred = model.predict(X_test)

In [16]:
predicted = np.argmax(y_pred, axis=1)

In [17]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8114134819922247


In [18]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

      Benign       0.84      0.96      0.89      2735
  Defacement       0.76      0.67      0.71      2499
     Malware       0.78      0.88      0.83      2834
    Phishing       0.74      0.75      0.75      2477
        Spam       0.93      0.78      0.85      3088

    accuracy                           0.81     13633
   macro avg       0.81      0.81      0.81     13633
weighted avg       0.81      0.81      0.81     13633



In [19]:
os.chdir("../")
os.chdir("models")
model.save("Model_v2.h5")
np.save('lblenc_v1.npy', encoder.classes_)
scalerfile = 'scaler_v1.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [24]:
import pathlib

In [21]:

model = ts.keras.models.load_model('Model_v2.h5')
converter = ts.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("tflite_model.tflite", "wb").write(tflite_model)

200608

In [23]:
model = ts.keras.models.load_model('Model_v2.h5')
converter = ts.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [ts.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [25]:
tflite_model_file = pathlib.Path('tflite_quant_model.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

55552